In [ ]:
import csv
import chars2vec
import re
import numpy as np
import keras.backend as K
from tensorflow.keras import datasets, layers, models, losses, callbacks
from utils import read_text_data, f1
from sklearn.model_selection import train_test_split
import statistics

Define a new import function as preprocessing isn't in this notebook

In [ ]:
def read_data_span(filename):
    """Reads csv file with python, text."""
    data = []
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        count = 0
        for row in reader:
            if row['span'] == '[]' or row['span'] == []:
                data.append([])
            else:
                data.append([int(j) for j in row['span'][1:-1].split(", ")])
    csvfile.close()
    return data

Read the test text and labels

In [ ]:
texts = read_text_data('data/tsd_test_readable.csv')
spans = read_data_span('data/tsd_test_readable.csv')

In [ ]:
##ADDED FOR PRESENTATION##

def add_to_dict(filename):
    with open(filename) as read_list:
    
        reader = csv.reader(read_list)
        for row in reader:
            toxic_dictionary[row[0]] = 1
    read_list.close()

Inspect the text

In [ ]:
texts

Inspect the spans

In [ ]:
spans

Load the test data into a numpy array

In [ ]:
test_X = np.zeros(shape=(len(texts), 1024, 50))
c2v_model = chars2vec.load_model('eng_50')
for x, string in enumerate(texts):
    for y, char in enumerate(string):
            char_vect = c2v_model.vectorize_words([char])
            test_X[x][y] = [word_vect for word_vect in char_vect[0]]

In [ ]:
example = np.zeros(shape=(1, 1024, 50))
test_string = ["You wanker, you fucking egit I hope you die you a**hole"]

for x, string in enumerate(test_string):
    for y, char in enumerate(string):
            char_vect = c2v_model.vectorize_words([char])
            example[x][y] = [word_vect for word_vect in char_vect[0]]

In [ ]:
model = models.load_model(f"DeconvNet_model_300_epochs")
y_pred = model.predict(test_X)

In [ ]:
def fix_word_boundaries(span, text):
    # "You fucking Moron you silly cunt" [6,7,8,9,10,11,12,13,14,15,16,28,29,30]
    # [4,5,6,7,8,9,10,11,12,13,14,15,16,28,29,30,31]
    seperated_text = []
    word = ''
    new_span = []
    current_word_span = []
    toxic_word = False
    for n, char in enumerate(text):
        if n in span:
            toxic_word = True
        if char == ' ':
            seperated_text.append(word)
            seperated_text.append(' ')
            word = ''
            if toxic_word:
                new_span.extend(current_word_span)
                current_word_span = []
                toxic_word = False
            else:
                current_word_span = []
                toxic_word = False
        else:
            word += char
            current_word_span.append(n)
        if n == len(text) - 1:
            seperated_text.append(word)
            if toxic_word:
                new_span.extend(current_word_span)
    return new_span

In [ ]:
scores = []
for x, pred in enumerate(y_pred):
    y_pred_f1_compatible = [j for j, i in enumerate(pred) if np.argmax(i) == 0]
    #y_pred_f1_compatible = fix_word_boundaries(y_pred_f1_compatible, texts[x])
    y_true_f1_compatible = spans[x]
    score = f1(y_pred_f1_compatible, y_true_f1_compatible)
    scores.append(score)

print('avg F1 %g' % statistics.mean(scores))


In [ ]:
test_pred = model.predict(example)

In [ ]:
for x, pred in enumerate(test_pred):
    char_arr = [j for j, i in enumerate(test_pred[0]) if np.argmax(i) == 0]
    print(f"text: {test_string[x]}")
    print(f"Predicted span: {char_arr}")
    print(f"Flagged text: {test_string[x][char_arr[0]:char_arr[-1]]}")
    

In [ ]:
##ADDED FOR PRESENTATION##

import csv
import pandas as pd

with open("data/tsd_train_readable.csv", encoding="utf-8", errors="ignore") as csv_file:
    with open("data/tsd_train_ground_truth_words.csv", "w", newline="") as out_file:
        reader = csv.reader(csv_file)
        writer = csv.writer(out_file)
        for row in reader:
            
            invalid_span = False
            complete_span = False
            string_vector = ""
            phrases = []
            if row[0] == "[]":
                offset_vector = []
                string_vector = row[1]
                complete_span = True
            else:
                offset_vector = row[0][1:-1].split(", ")
                offset_vector_int = []
                for item in offset_vector:
                    if item.isnumeric():
                        offset_vector_int.append(int(item))
                    else:
                        invalid_span = True
                        break
                offset_vector = offset_vector_int
                for string_vector_index, char_index in enumerate(offset_vector):
                    if invalid_span:
                        break
                    if complete_span:
                        break
                    if char_index > len(row[1])-1:
                        break
                    if string_vector_index == 0:
                        string_vector = string_vector + row[1][int(char_index)]
                    else:
                        if int(char_index) != offset_vector[string_vector_index-1] + 1:
                            if len(string_vector.split(" ")) < 2:
                                phrases.append(string_vector)
                                string_vector = ""
                                string_vector = string_vector + row[1][int(char_index)]
                        else:
                            string_vector = string_vector + row[1][int(char_index)]
                if len(string_vector.split(" ")) < 2:
                    phrases.append(string_vector)
                if complete_span == False and string_vector != "" and phrases != []:
                    for phrase in phrases:
                        print(phrase)
                        writer.writerow([phrase])
csv_file.close()
out_file.close()

In [ ]:
##ADDED FOR PRESENTATION##
toxic_dictionary = dict({})
add_to_dict("data/tsd_train_ground_truth_words.csv")

test = read_text_data("data/tsd_test_readable.csv")

In [ ]:
##ADDED FOR PRESENTATION##
scores = []
for x, text in enumerate(test):
    predict_spans = []
    for phrase in toxic_dictionary.keys():
        if text.find(phrase) != -1:
            predict_spans.extend(range(text.find(phrase), text.find(phrase) + len(phrase)))
            break
    predict_spans = list(set(predict_spans))
    #ensemble = set(predict_spans).intersection([j for j, i in enumerate(y_pred[x]) if np.argmax(i) == 0])
    #score = f1(ensemble, spans[x])
    #uncomment the comment out comment below to test ensemble
    score = f1(predict_spans, spans[x])
    scores.append(score)

print('avg F1 %g' % statistics.mean(scores))